<a href="https://colab.research.google.com/github/shivaprasad909/notebook_elephant/blob/main/Mining_Seed_Collin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

# @title Step 1: Upload files
#Upload .env & oracle-keystore.json

In [1]:
# @title Step 2: Prepare
# @title  {"vertical-output":true}

import csv
import json
import os
from urllib.parse import urlparse, parse_qs

parcel_id = "2595533" # @param {"type":"string"}
address = "1033 ACACIA DR, ANNA, TX, 75409-5001" # @param {"type":"string"}
request_method = "GET" # @param {"type":"string"}
url = "https://esearch.collincad.org/Property/View/2595533?year=2025" # @param {"type":"string"}
County = "Collin" # @param {"type":"string"}
headers = "" # @param {"type":"string"}
json_body = "" # @param {"type":"string"}

fieldnames = [
    "parcel_id",
    "address",
    "method",
    "url",
    "multiValueQueryString",
    "county",
    "json",
    "headers",
    "source_identifier",
]


def extract_base_url_and_params(url):
    parsed = urlparse(url)
    base_url = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"

    # Always store params as lists
    params = {}
    if parsed.query:
        for k, v in parse_qs(parsed.query, keep_blank_values=True).items():
            params[k] = v  # always a list

    # Include fragment in base_url; also parse ?query that might be inside the fragment
    if parsed.fragment:
        frag_path, _, frag_query = parsed.fragment.partition('?')
        # Keep the path-like part of the fragment in the base URL
        base_url = f"{base_url}#{frag_path}" if frag_path else f"{base_url}#"

        if frag_query:
            for k, v in parse_qs(frag_query, keep_blank_values=True).items():
                if k in params:
                    # Merge with existing values
                    params[k].extend(v)
                else:
                    params[k] = v  # always a list

    return base_url, params


def process_input_data():
  base_url, params = extract_base_url_and_params(url)

  row = {
      "parcel_id": parcel_id,
      "address": address,
      "method": request_method,
      "url": base_url,
      # Important: multiValueQueryString must be JSON string with array values
      "multiValueQueryString": json.dumps(params, ensure_ascii=False) if params else "",
      "county": County,
      "json": json_body,
      "headers": headers,
      "source_identifier": parcel_id,
  }



  with open("seed.csv", "w", newline="", encoding="utf-8") as f:
      writer = csv.DictWriter(f, fieldnames=fieldnames)
      writer.writeheader()
      writer.writerow(row)


process_input_data()
print(f"✅ Prepare done for parcel ID {parcel_id}")


✅ Prepare done for parcel ID 2595533


In [2]:

# @title Step 2: Transform
from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv
import os


def has_submit_errors(path="submit_errors.csv"):
    if not os.path.exists(path):
      return False
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_transform():
    try:
        subprocess.run(
            [
              "npx", "-y", "@elephant-xyz/cli@latest", "transform", "--legacy-mode",
              "--group", "seed",
              "--input-csv", "seed.csv",
              "--output-zip", "transformed_seed.zip",
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Transform failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        print("✅ Transform done\n")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_transform()



✅ Transform done



In [3]:
# @title Step 3: Validate
!pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_validate():
    try:
        result = subprocess.run(
            ["npx", "-y", "@elephant-xyz/cli@latest", "validate", "transformed_seed.zip"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        print("STDOUT:\n", result.stdout)
        print("STDERR:\n", result.stderr)

        if result.returncode != 0:
            print(f"❌ Validate failed with exit code {result.returncode}")
        else:
            print("✅ Validate done")

    except Exception as e:
        print("Unexpected error:", str(e))


if __name__ == "__main__":
    run_validate()


STDOUT:
 🐘 Elephant Network CLI - Validate (Single Property)


✅ Validation process finished

📊 Validation Report:
  Total files scanned:    1
  Files skipped:          0
  Validation errors:      0
  Successfully validated: 1
  Total files handled:    1
  Duration:               0s

✅ All files passed validation!

STDERR:
 
✅ Validate done


In [ ]:
!cat submit_errors.csv


property_cid,data_group_cid,file_path,error_path,error_message,timestamp


In [4]:
# @title Step 4: Hash
!pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def get_seed_cid(path="seed-results.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row["propertyCid"]


def run_hash():
    try:
        subprocess.run(
            [
                "npx", "-y", "@elephant-xyz/cli@latest",
                "hash", "transformed_seed.zip",
                "--output-zip", "hashed-data.zip",
                "--output-csv", "seed-results.csv",
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Validate failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        seed_group_cid = get_seed_cid()
        print("✅ Hash done\n")
        print(f"Seed group CID: {seed_group_cid}\n")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_hash()



✅ Hash done

Seed group CID: bafkreigunu67pmpdmdjmyzu3t77r6zf3e2nat4hnuveuevoocrokb2in2q



In [ ]:
# Seed group CID: bafkreigwnow7touct47orqzldeg6cn2ft27kuodagieidr7hmlk25s3nkq

In [10]:
# @title Step 5: Upload
! pip3 install python-dotenv requests -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv

import requests


def get_seed_info(path="seed-results.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def count_uploaded_files(output_dir="output"):
    """
    Count all files recursively in the output directory and its subdirectories.
    """
    if not os.path.exists(output_dir):
        return 0

    file_count = 0
    for root, dirs, files in os.walk(output_dir):
        file_count += len(files)
    return file_count


def collect_data_ipfs_links(data_cid):
  seed_data_link = f"https://ipfs.io/ipfs/{data_cid}"

  seed_data = requests.get(seed_data_link).json()

  property_seed_cid = seed_data["relationships"]["property_seed"]["/"]
  property_seed_link = f"https://ipfs.io/ipfs/{property_seed_cid}"

  property_seed_data = requests.get(property_seed_link).json()

  property_cid, address_cid = property_seed_data["from"]["/"], property_seed_data["to"]["/"]

  property_link = f"https://ipfs.io/ipfs/{property_cid}"
  address_link = f"https://ipfs.io/ipfs/{address_cid}"

  return seed_data_link, property_seed_link, property_link, address_link


def run_validate_and_upload():
    try:
        subprocess.run(
            ["npx", "-y", "@elephant-xyz/cli@1.28.4", "upload", "hashed-data.zip", "--output-csv", "seed-results.csv"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Validate failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        seed_info = get_seed_info()
        seed_group_cid, data_cid, html_link = seed_info["dataGroupCid"], seed_info["dataCid"], seed_info["htmlLink"]

        files_uploaded = count_uploaded_files()

        data_ipfs_links = collect_data_ipfs_links(data_cid)
        seed_group_link, property_seed_link, property_link, address_link = data_ipfs_links

        print("✅ Upload done\n")
        print(f"{files_uploaded} files uploaded\n")

        print(f"Seed group CID: {seed_group_cid}\n")
        print(f"HTML link: {html_link}\n")

        print(f"Seed group IPFS link: {seed_group_link}")
        print(f"relationship IPFS link: {property_seed_link}")
        print(f"property_seed IPFS link: {property_link}")
        print(f"unnormalized_address IPFS link: {address_link}")


        print("✅ Upload done\n")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_validate_and_upload()


✅ Upload done

4 files uploaded

Seed group CID: bafkreibeubpcioeeakky5vpanyjcfn3fl6nla22sdr47q3yesrfi6i7mmm

HTML link: https://ipfs.io/ipfs/bafybeic43k6evko3ckzuyh5hgt2lk3k2upofk67lfvaunkbrrelqtfedym

Seed group IPFS link: https://ipfs.io/ipfs/bafkreigunu67pmpdmdjmyzu3t77r6zf3e2nat4hnuveuevoocrokb2in2q
relationship IPFS link: https://ipfs.io/ipfs/bafkreigsphzxlkml7w4drpkfzmmntwcbwjrx4sk54zbrxz7xtfvpdyt54u
property_seed IPFS link: https://ipfs.io/ipfs/bafkreih4gfyfw3r4rpsmgynjno52v46xy7zzvrxgwyfxgnfg6pt45mprwq
unnormalized_address IPFS link: https://ipfs.io/ipfs/bafkreiffej7jw3rrr4ilgykfvpjkljjb7emzw2k7ee6f4zw76sndpbxqh4
✅ Upload done



In [12]:
# @title Step 6: Submit

! pip3 install python-dotenv -q

from dotenv import load_dotenv
load_dotenv()

import subprocess
import sys
import csv


def get_transaction_hash(path="transaction-status.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader, None)
        if first_row is None:
            raise ValueError("CSV file is empty")
        return first_row["transactionHash"]


def has_submit_errors(path="submit_errors.csv"):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return next(reader, None) is not None


def run_submit_to_contract():
    try:
        subprocess.run(
            [
                "npx", "-y", "@elephant-xyz/cli@latest", "submit-to-contract", "seed-results.csv",
                "--keystore-json", "./keystore.json",
                "--keystore-password", "almiT@Block#hain",
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            check=True,
            text=True,
        )
        if has_submit_errors():
            print("❌ Submit failed, please check submit_errors.csv for details", file=sys.stderr)
            return

        transaction_hash = get_transaction_hash()
        transaction_link = f"https://polygonscan.com/tx/{transaction_hash}"

        print("✅ Submit done\n")
        print(f"Transaction link: {transaction_link}")

    except subprocess.CalledProcessError as e:
        print(f"Command failed (exit code {e.returncode}):", file=sys.stderr)
        print(e.stderr.strip(), file=sys.stderr)
        sys.exit(e.returncode)


if __name__ == "__main__":
    run_submit_to_contract()


✅ Submit done

Transaction link: https://polygonscan.com/tx/0x78a1060bba3fa8ef38d70f96fa62ac487bd736bbcf22548d602cff049e93c572


In [13]:

# @title Step 6: Construct seed_output.zip and Download it
!mkdir -p transformed_seed
!unzip -q transformed_seed.zip -d transformed_seed

!mkdir -p seed_output
!cp ./transformed_seed/*/property_seed.json ./transformed_seed/*/unnormalized_address.json ./seed-results.csv ./seed.csv seed_output/
!zip -q -r seed_output.zip seed_output


import os; from google.colab import files; (files.download('seed_output.zip'), print("✅ File was downloaded successfully"))[1] if os.path.exists('seed_output.zip') else print("❌ File not found")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ File was downloaded successfully
